<a href="https://colab.research.google.com/github/felipeoliveiralps/Analise-Sentimento-Ecommerce/blob/master/notebooks/NLP/BERT_PositivoNegativo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.0 Carregando dataset**

In [ ]:
import pandas as pd
dataSemStopWords = pd.read_csv('../../datasets/dataSemStopWords.csv', engine='python')


**2.0 Estamos com um Problema! (Positivo vs. Negativo)**

* Como ja visto, o dataset que pré-processei anteriormente contém a regra de negócio completa, com as três classes de sentimento: positivo, neutro e negativo. No entanto, o objetivo deste notebook específico é treinar um modelo BERT focado em uma análise de sentimento bipolar, ou seja, capaz de diferenciar apenas comentários claramente positivos dos negativos.

* Para garantir que o modelo aprenda os padrões distintivos dessas duas classes sem a interferência ou o "ruído" dos comentários neutros, irei removê-los completamente. Essa simplificação do problema permite que o modelo se especialize em sua tarefa, potencializando sua performance na distinção dos dois polos de sentimento.

* O código a seguir executa essa filtragem, excluindo todas as avaliações com nota 3. Adicionalmente, ele converte a coluna de texto para o tipo string para garantir a compatibilidade com as ferramentas de modelagem.

In [ ]:
dataSemStopWords['feedback'] = dataSemStopWords.apply(lambda x: 1 if x['rating'] >= 3 else 0, axis=1)


In [ ]:
dataSemStopWords = dataSemStopWords[dataSemStopWords['rating'] != 3]

In [ ]:
dataSemStopWords['texto_lematizado'] = dataSemStopWords['texto_lematizado'].astype(str)


In [ ]:
dataSemStopWords

In [ ]:
pip install tensorflow

**3.0 Tokenização!!**

* Para manter o modelo eficiente e focado, limitei o vocabulário às 15.000 palavras mais comuns, e qualquer palavra fora desse conjunto será tratada como um token desconhecido (<unk>). Após a tokenização, cada avaliação se torna uma sequência de números, mas com comprimentos diferentes, além de um padding de 150 token por avaliação.



In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Instanciar e configurar o tokenizador
tokenizer = Tokenizer(num_words=15000, oov_token="<unk>")  # Limitar o vocabulário aos 15000 termos mais frequentes

# 2. Ajustar o tokenizador aos seus textos (coluna texto_lematizado)
tokenizer.fit_on_texts(dataSemStopWords['texto_lematizado'])

# 3. Converter os textos em sequências de inteiros
sequences = tokenizer.texts_to_sequences(dataSemStopWords['texto_lematizado'])

# 4. Padronizar o tamanho das sequências (opcional, mas recomendado)
max_length = 150  # Tamanho máximo da sequência (ajuste conforme necessidade)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# 5. Visualizar o resultado
print(f"Tamanho do vocabulário: {len(tokenizer.word_index)}")
print(f"Forma do array de sequências: {padded_sequences.shape}")
print(f"Exemplo de sequência original: {sequences[0][:20]}...")
print(f"Exemplo de sequência padronizada: {padded_sequences[0][:20]}...")


**3.1 Divisão Estratificada: Treino, Validação e Teste**

* Utilizarei uma separação em três partes: 80% para treino, 10% para validação (ajuste de hiperparâmetros) e 10% para teste (avaliação final e imparcial).


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# --- Passo 1: Primeira Divisão (Treino / Resto) ---
train_df, temp_df = train_test_split(
    dataSemStopWords,
    test_size=0.2,
    random_state=42,
    stratify=dataSemStopWords['feedback']
)

# --- Passo 2: Segunda Divisão (Validação / Teste) ---
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df['feedback'] # Estratificar aqui também é importante
)

print(dataSemStopWords)
print(train_df)
print(val_df)
print(test_df)


In [ ]:

from transformers import AutoTokenizer, TFBertForSequenceClassification, create_optimizer
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
import matplotlib.pyplot as plt
import os # Para gerenciar caminhos de arquivo





**Bertimbau!**

In [ ]:
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

**Preparando dados**
* Para preparar os dados, vou usar o tokenizer para converter cada avaliação em input IDs e attention masks, padronizando o comprimento em 150 tokens. Em seguida, encapsularei esses dados tokenizados junto com seus respectivos rótulos em objetos tf.data.Dataset, criando um pipeline de dados otimizado e de alta performance para o treinamento no TensorFlow.

In [ ]:
train_encodings = tokenizer(train_df['texto_lematizado'].tolist(), truncation=True, padding=True, max_length=150, return_tensors='tf')
val_encodings = tokenizer(val_df['texto_lematizado'].tolist(), truncation=True, padding=True, max_length=150, return_tensors='tf')
test_encodings = tokenizer(test_df['texto_lematizado'].tolist(), truncation=True, padding=True, max_length=150, return_tensors='tf')

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_df['feedback'].values))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_df['feedback'].values))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_df['feedback'].values))


**Class Weights para o desbalanceamento dos dados**

In [ ]:
y_train_labels = train_df['feedback'].values
class_labels = np.unique(y_train_labels)
class_weights_array = compute_class_weight(class_weight='balanced', classes=class_labels, y=y_train_labels)
class_weight_dict = dict(zip(class_labels, class_weights_array))


print(f"Classes: {class_labels}")
print(f"Pesos das Classes Calculados: {class_weights_array}")
print(f"Dicionário de Pesos para Keras: {class_weight_dict}")


**HiperParametros**
* Com os dados prontos, agora vou configurar os parametros para a modelagem. Irei configurar o batch_size como 16, que define quantos exemplos o modelo verá antes de atualizar seus pesos. O número de epochs será 2, o que significa que o modelo irá percorrer todo o conjunto de dados de treino duas vezes, além do learning rate scheduler com warmup.


In [ ]:
batch_size = 16
epochs = 2

train_batches = train_dataset.shuffle(len(train_df)).batch(batch_size)
val_batches = val_dataset.batch(batch_size)
test_batches = test_dataset.batch(batch_size)

steps_per_epoch = len(train_df) // batch_size
if len(train_df) % batch_size != 0:
    steps_per_epoch += 1
total_train_steps = steps_per_epoch * epochs

optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=total_train_steps,
    num_warmup_steps=int(0.1 * total_train_steps),
    weight_decay_rate=0.01
)

**Compilando o modelo!!**

In [ ]:
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

**Treinando**

In [ ]:
print("Iniciando o treinamento do BERTimbau com class_weight...")

history = model.fit(
    train_batches,
    validation_data=val_batches,
    epochs=epochs,
    class_weight=class_weight_dict,
    verbose=1
)
print("Treinamento concluído!")